In [1]:
# Importamos Spark
from pyspark.sql import SparkSession

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1606494795592_0006,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Creamos la instancia de Spark
spark=SparkSession.builder.appName('data_processing').getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Cargamos los datos de S3
df=spark.read.csv('s3://aws-logs-247500789477-us-east-1/datasets/Casos_positivos_de_COVID-19_en_Colombia.csv',inferSchema=True,header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Vemos que columnas tiene
df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['ID de caso', 'Fecha de notificación', 'Codigo DIVIPOLA', 'Ciudad de ubicación', 'Departamento o Distrito ', 'atención', 'Edad', 'Sexo', 'Tipo', 'Estado', 'País de procedencia', 'FIS', 'Fecha de muerte', 'Fecha diagnostico', 'Fecha recuperado', 'fecha reporte web']

In [5]:
# Cuantas columnas tiene
len(df.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

16

In [6]:
# Cuantos datos tiene
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6211

In [7]:
# Cual es la dimensión de el dataframe
print((df.count(),len(df.columns)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(6211, 16)

In [8]:
# Cual es el esquema
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ID de caso: integer (nullable = true)
 |-- Fecha de notificación: timestamp (nullable = true)
 |-- Codigo DIVIPOLA: integer (nullable = true)
 |-- Ciudad de ubicación: string (nullable = true)
 |-- Departamento o Distrito : string (nullable = true)
 |-- atención: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Tipo: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- País de procedencia: string (nullable = true)
 |-- FIS: string (nullable = true)
 |-- Fecha de muerte: string (nullable = true)
 |-- Fecha diagnostico: timestamp (nullable = true)
 |-- Fecha recuperado: string (nullable = true)
 |-- fecha reporte web: timestamp (nullable = true)

In [9]:
# Vemos los primeros 5 registros
df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+------+-------------------+--------------------+---------------+-------------------+--------------------+-------------------+
|ID de caso|Fecha de notificación|Codigo DIVIPOLA|Ciudad de ubicación|Departamento o Distrito |  atención|Edad|Sexo|       Tipo|Estado|País de procedencia|                 FIS|Fecha de muerte|  Fecha diagnostico|    Fecha recuperado|  fecha reporte web|
+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+------+-------------------+--------------------+---------------+-------------------+--------------------+-------------------+
|         1|  2020-03-02 00:00:00|          11001|        Bogotá D.C.|             Bogotá D.C.|Recuperado|  19|   F|  Importado|  Leve|             Italia|2020-02-27T00:00:...|          -   -|2020-03-06 00:00:00|2020-03-13T00:00:...|2020-

In [10]:
# Vemos solamente la edad, el genero, y el país de procedencia de los primeros 5 registros
df.select('Edad','Sexo', 'País de procedencia').show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----+-------------------+
|Edad|Sexo|País de procedencia|
+----+----+-------------------+
|  19|   F|             Italia|
|  34|   M|             España|
|  50|   F|             España|
|  55|   M|           Colombia|
|  25|   M|           Colombia|
+----+----+-------------------+
only showing top 5 rows

In [12]:
# Vemos más datos acerca de la edad, un total de 6211 registros, con edad promedio de 41 años y desviación estándar de 18 años
df.describe().select('Edad').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+
|              Edad|
+------------------+
|              6211|
| 41.23715987763645|
|18.468991567406906|
|                 1|
|               103|
+------------------+

In [18]:
# Vemos que posibilidades hay en atención
df.select('atención').distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+
|    atención|
+------------+
|  Recuperado|
|Hospital UCI|
|        Casa|
|   Fallecido|
|    Hospital|
|         N/A|
|        casa|
+------------+

In [14]:
# Contamos cuantos recuperados hay
df.filter(df['atención']=='Recuperado').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1411

In [15]:
# Contamos cuantos fallecidos hay
df.filter(df['atención']=='Fallecido').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

278

In [20]:
# Vemos de que países proceden los casos y cuantos por país
df.groupBy('País de procedencia').count().orderBy('count',ascending=False).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
| País de procedencia|count|
+--------------------+-----+
|            Colombia| 5401|
|              España|  258|
|Estados Unidos de...|  206|
|             Ecuador|   47|
|              México|   46|
|              Panamá|   34|
|              Brasil|   30|
|             Turquía|   28|
|             Jamaica|   16|
|             Francia|   15|
|         Reino Unido|   14|
|         Puerto Rico|   13|
|              Egipto|   11|
|            Alemania|   10|
|República Dominicana|    9|
|              Italia|    9|
|                Perú|    9|
|San Martín (Francia)|    7|
|              Israel|    5|
|                Cuba|    4|
+--------------------+-----+
only showing top 20 rows

In [31]:
# Función para determinar si el paciente es menor o mayor de edad
def patient_type(Edad):
    if Edad < 18:
        return 'Menor'
    else:
        return 'Adulto'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
# Importamos funciones y tipos de datos SQL
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType,DoubleType,IntegerType

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
# Creamos la columna tipo de paciente utilizando la función creada anteriormente, en un nuevo dataframe
type_udf=udf(patient_type,StringType())

df2 = df.withColumn('Tipo paciente',type_udf(df['Edad']))
df2.show(10,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+------+-------------------------+-----------------------+---------------+-------------------+-----------------------+-------------------+-------------+
|ID de caso|Fecha de notificación|Codigo DIVIPOLA|Ciudad de ubicación|Departamento o Distrito |atención  |Edad|Sexo|Tipo       |Estado|País de procedencia      |FIS                    |Fecha de muerte|Fecha diagnostico  |Fecha recuperado       |fecha reporte web  |Tipo paciente|
+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+-----------+------+-------------------------+-----------------------+---------------+-------------------+-----------------------+-------------------+-------------+
|1         |2020-03-02 00:00:00  |11001          |Bogotá D.C.        |Bogotá D.C.             |Recuperado|19  |F   |Importado  |Leve  |Italia                   

In [34]:
# Que tipos hay
df.select('Tipo').distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+
|       Tipo|
+-----------+
|  Importado|
| En Estudio|
| En estudio|
|Relacionado|
|relacionado|
+-----------+

In [51]:
# Borramos la columna tipo
df_new=df2.drop('Tipo')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
# Vemos nuestro dataframe que ahora incluye tipo de paciente y excluye tipo
df_new.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+------+-------------------+--------------------+---------------+-------------------+--------------------+-------------------+-------------+
|ID de caso|Fecha de notificación|Codigo DIVIPOLA|Ciudad de ubicación|Departamento o Distrito |  atención|Edad|Sexo|Estado|País de procedencia|                 FIS|Fecha de muerte|  Fecha diagnostico|    Fecha recuperado|  fecha reporte web|Tipo paciente|
+----------+---------------------+---------------+-------------------+------------------------+----------+----+----+------+-------------------+--------------------+---------------+-------------------+--------------------+-------------------+-------------+
|         1|  2020-03-02 00:00:00|          11001|        Bogotá D.C.|             Bogotá D.C.|Recuperado|  19|   F|  Leve|             Italia|2020-02-27T00:00:...|          -   -|2020-03-06 00:00:00|2020-03-13T00:00:...|2020-03-06 

In [53]:
# Función para eliminar espacios y mayusculas de los nombres de las columnas
for col in df_new.columns:
    df_new = df_new.withColumnRenamed(col, col.replace(" ", "").lower())
df_new.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['iddecaso', 'fechadenotificación', 'codigodivipola', 'ciudaddeubicación', 'departamentoodistrito', 'atención', 'edad', 'sexo', 'estado', 'paísdeprocedencia', 'fis', 'fechademuerte', 'fechadiagnostico', 'fecharecuperado', 'fechareporteweb', 'tipopaciente']

In [60]:
# Dirección para guardar los resultados csv
write_uri='s3://aws-logs-247500789477-us-east-1/datasets/covid_csv'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
# Guardar los resultados csv
df_new.coalesce(1).write.format("csv").option("header","true").save(write_uri)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
# Dirección para guardar los resultados parquet
parquet_uri='s3://aws-logs-247500789477-us-east-1/datasets/covid_parquet'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
# Guardar los resultados parquet
df_new.write.format('parquet').save(parquet_uri)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…